# Quickstart AltaiPony

Let's first get the base class and the IO function we want to use:

In [ ]:
from altaipony.flarelc import FlareLightCurve
from altaipony.lcio import from_K2SC_source

Now download a **K2SC** light curve from archive and create a ``FlareLightCurve``.

(It's enough to pass the EPIC ID to ``from_K2SC_source``, but you could pass a path instead as well.)

In [ ]:
EPIC = 211119999
flc = from_K2SC_source(EPIC)

Let's verify that we succeeded:

In [ ]:
flc

The representation tells us that we have indeed created a ``FlareLightCurve``.

Now let's look at some cool tools we inherited from the awesome ``lightkurve`` package:

In [ ]:
flc.plot()

This is the raw light curve. However, the **K2SC** light curve we just fetched has de-trended flux as well:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12,6))
plt.plot(flc.time, flc.detrended_flux, c='k')
plt.plot(flc.time, flc.flux, c='c')

**K2SC** preserved flare signal fairly well, so this is what we use for flare finding.

So let us find the flares in this light curve (one, at least, is pretty obvious, right?)

In [ ]:
flc = flc.find_flares()
flc.flares

AHA! Four flares! Let's see where they are:

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(flc.time, flc.detrended_flux, c='k', label='detrended flux')
plt.plot(flc.time, flc.it_med, c='c', label='median')
for i,f in flc.flares.iterrows():
    plt.plot(flc.time[f.istart:f.istop+1], flc.detrended_flux[f.istart:f.istop+1], c='r')
plt.ylim(39500,43000)
plt.legend()

No surprise we found the big one. The other three are not that obvious.

Now, what if doubt that these three are real? 

What if it's just outliers and similar sized flares would be lost in the noise? 

Or rest undiscovered due to low time resolution?

Let us test that (may take a moment):

In [ ]:
flc = flc.characterize_flares(iterations=100)
flc.flares

The large event is relatively likely a flare as we would expect it (1st output line). Flares with similar equivalent durations (ED)² as the bigger flare are recovered in ~80% of the cases. 

The small one is lost in the noise and is most likely an outlier that happens to fit the detection criteria¹ (2nd output line). 

We have run an injection recovery procedure, that introduces synthetic flares similar to the one we recovered and tries to find them again. 

The ``flares`` attribute stores the relevant information about each flare as soon as it is found. When we run the ``characterize_flares`` method on our ``FlareLightCurve``, we add the ``ed_rec_corr`` and ``rec_prob``  attributes. ``ed_rec_corr`` gives the ratio between the injected flare's ED and the recovered one in a bin around the observed value (here: 1601 s). ``rec_prob`` is the number of recovered flares relative to all injected ones, or recovery probability - again in a ED bin around the observed flare.

¹detection criteria are listed in the ``find_flares_in_cont_obs_period`` function in the ``altai`` module

²equivalent duration is a proxy for flare energy that is relative to total stellar flux. It is measured in seconds: It is how long the star needs to quietly shine until it has released just as much flux as the flare did.

We can look at what happens, when we inject and recover synthetic flares within **AltaiPony**:

In [ ]:
data, fake_lc = flc.sample_flare_recovery(iterations=10)

``fake_lc`` stores the flux with injected flare event after the last iteration: 

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(fake_lc.time, fake_lc.detrended_flux, c='k')
plt.plot(fake_lc.time, fake_lc.it_med, c='c')
for i,f in fake_lc.flares.iterrows():
    plt.plot(fake_lc.time[f.istart-10:f.istop+50], fake_lc.detrended_flux[f.istart-10:f.istop+50], c='r')

``data`` stores all injected flares, if they were recovered, and if yes, also the recovered ED, just as with real flares.

In [ ]:
data.head()

Flares that overlap are summed up and treated as a single complex flare and the recovered ED is compared to the sum of injected EDs. 